In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
with open('data/my_val.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)
del data

with open('data/one-vs-all.pkl','rb') as f:
    data = joblib.load(f)
locals().update(data)
del data

In [3]:
def stacking(x, my_models):
    stack_x = np.zeros((len(x), len(my_models)))
    for i, m in enumerate(my_models):
        p = m.predict_proba(x)[:, 1]
        stack_x[:, i] = p
    return stack_x

In [4]:
st_x = stacking(x, my_models)
st_val_x = stacking(val_x, my_models)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
pipe = Pipeline([('전처리',StandardScaler()), ('알고리즘', KNeighborsClassifier())])

In [7]:
param = [
    {'알고리즘':[RandomForestClassifier()],
     '전처리': [MinMaxScaler()],
     '알고리즘__n_estimators': [100],
     '알고리즘__criterion': ['entropy'],
     '알고리즘__max_depth': [None]+list(np.arange(10,21))}]

In [8]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=pipe, param_grid=param,
                    scoring='accuracy', cv=4, n_jobs=-1, verbose=1)
grid.fit(st_x,y)

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   25.6s finished


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('전처리', StandardScaler()),
                                       ('알고리즘', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid=[{'알고리즘': [RandomForestClassifier(criterion='entropy')],
                          '알고리즘__criterion': ['entropy'],
                          '알고리즘__max_depth': [None, 10, 11, 12, 13, 14, 15, 16,
                                              17, 18, 19, 20],
                          '알고리즘__n_estimators': [100],
                          '전처리': [MinMaxScaler()]}],
             scoring='accuracy', verbose=1)

In [9]:
accuracy_score(val_y, grid.best_estimator_.predict(st_val_x))

0.1474487746082764